In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import argparse


Transform = transforms.Compose([
    transforms.RandomCrop(32, padding = 4),
    transforms.RandomHorizontalFlip(p = 0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean = (0.4914, 0.4822, 0.4465), std = (0.2023, 0.1994, 0.2010))
])

TrainDataset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = Transform)
TrainLoader = torch.utils.data.DataLoader(TrainDataset, batch_size = 128, shuffle = True, num_workers = 2)

TestDataset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = Transform)
TestLoader = torch.utils.data.DataLoader(TestDataset, batch_size = 100, shuffle = False, num_workers = 2)

# Define the ResNet-18 model
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size = 3, stride = stride, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size = 3, stride = 1, padding = 1, bias = False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size = 1, stride = stride, bias = False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

class ResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet18, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        self.layer1 = self.make_layer(BasicBlock, 64, 2, stride=1)
        self.layer2 = self.make_layer(BasicBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(BasicBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(BasicBlock, 512, 2, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * BasicBlock.expansion, num_classes)

    def make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)

        return out

# Define the hyperparameters
learning_rate = 0.1
momentum = 0.9
weight_decay = 5e-4
num_epochs = 10

Files already downloaded and verified
Files already downloaded and verified


In [14]:
# Define command line arguments
parser = argparse.ArgumentParser(allow_abbrev=False, add_help=False)
parser.add_argument('--data_path', type=str, default='./data',
                    help='Path to the dataset directory')
parser.add_argument('--num_workers', type=int, default=2,
                    help='Number of data loader workers')
parser.add_argument('--optimizer', type=str, default='sgd',
                    help='Optimizer for training (sgd, adam, etc.)')
parser.add_argument('--use_cuda', type=bool, default=False,
                    help='Whether to use CUDA for training')
parser.add_argument('--batch_size', type=int, default=128,
                    help='Training batch size')
parser.add_argument('--test_batch_size', type=int, default=100,
                    help='Test batch size')
parser.add_argument('--learning_rate', type=float, default=0.1,
                    help='Learning rate for optimizer')
parser.add_argument('--momentum', type=float, default=0.9,
                    help='Momentum for optimizer')
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay for optimizer')
parser.add_argument('--num_epochs', type=int, default=5,
                    help='Number of epochs to train for')
parser.add_argument('--KernelArg', nargs='*', help='Kernel arguments')
args, unknown = parser.parse_known_args()

# Parse kernel arguments
if args.KernelArg is not None:
    unknown += args.KernelArg
args = parser.parse_args(unknown)

# Set device
device = torch.device('cuda' if args.use_cuda and torch.cuda.is_available() else 'cpu')

usage: ipykernel_launcher.py [--data_path DATA_PATH]
                             [--num_workers NUM_WORKERS]
                             [--optimizer OPTIMIZER] [--use_cuda USE_CUDA]
                             [--batch_size BATCH_SIZE]
                             [--test_batch_size TEST_BATCH_SIZE]
                             [--learning_rate LEARNING_RATE]
                             [--momentum MOMENTUM]
                             [--weight_decay WEIGHT_DECAY]
                             [--num_epochs NUM_EPOCHS]
                             [--KernelArg [KERNELARG ...]]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/karanvora/.local/share/jupyter/runtime/kernel-b7fcc814-b389-4ed6-b657-6ba52c8b5fe9.json


SystemExit: 2

In [3]:
string = 'TRue'

print(bool(string))

True


In [2]:
command = 'Hello'

match command.lower():
    case 'hello':
        print('Hello')
    case 'hi':
        print('Hi')

Hello
